<a href="https://colab.research.google.com/github/vishal-burman/PyTorch-Architectures/blob/master/modeling_DenseNet121/test_sample_DenseNet121.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! git clone https://github.com/vishal-burman/PyTorch-Architectures.git

In [ ]:
%cd PyTorch-Architectures/modeling_DenseNet121/

/content/PyTorch-Architectures/modeling_DenseNet121


In [11]:
import os
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Subset

from torchvision import datasets
from torchvision import transforms

from model import DenseNet121

if torch.cuda.is_available():
  torch.backends.cudnn.deterministic = True

In [ ]:
###################
# Model Settings
###################

# Hyperparameters
RANDOM_SEED = 1
LEARNING_RATE = 0.001
BATCH_SIZE = 128
NUM_EPOCHS = 20

# Architecture
NUM_CLASSES = 10

# Other
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
grayscale = False

In [ ]:
####################
# CIFAR-10 Dataset
####################

# training samples ~ 48000 samples
train_indices = torch.arange(0, 48000)
# validation samples ~ 2000 samples
valid_indices = torch.arange(48000, 50000)

train_and_valid = datasets.CIFAR10(root='data',
                                   train=True,
                                   transform=transforms.ToTensor(),
                                   download=True)

train_dataset = Subset(train_and_valid, train_indices)
valid_dataset = Subset(train_and_valid, valid_indices)
test_dataset = datasets.CIFAR10(root="data",
                                train=False,
                                transform=transforms.ToTensor(),
                                download=False)

train_loader = DataLoader(dataset=train_dataset,
                          batch_size=BATCH_SIZE,
                          num_workers=4,
                          shuffle=True)

valid_loader = DataLoader(dataset=valid_dataset,
                          batch_size=BATCH_SIZE,
                          num_workers=4,
                          shuffle=False)

test_loader = DataLoader(dataset=test_dataset,
                         batch_size=BATCH_SIZE,
                         num_workers=4,
                         shuffle=False)

In [ ]:
# Sanity check the loaders
torch.manual_seed(0)

for epoch in range(2):
  for batch_idx, (features, label) in enumerate(train_loader):

    print("Epoch: %d | Batch: %d | Batch Size: %d" % (epoch+1, batch_idx, label.size()[0]))

    features = features.to(device)
    label = label.to(device)
    break

Epoch: 1 | Batch: 0 | Batch Size: 128
Epoch: 2 | Batch: 0 | Batch Size: 128


In [ ]:
# Sanity check shuffling
# label indices should be in different order
# label order should be different in second epoch
for images, labels in train_loader:
  pass
print(labels[:10])

for images, labels in train_loader:
  pass
print(labels[:10])

tensor([3, 0, 5, 4, 8, 2, 1, 7, 5, 4])
tensor([5, 9, 3, 0, 6, 4, 7, 2, 2, 1])


In [ ]:
# Sanity check ~ validation and test set should be diverse
# ~ should contain all classes

for images, labels in valid_loader:
  pass
print(labels[:10])

for images, labels in test_loader:
  pass
print(labels[:10])

tensor([5, 0, 3, 6, 8, 7, 9, 5, 6, 6])
tensor([7, 5, 8, 0, 8, 2, 7, 0, 3, 5])


In [13]:
torch.manual_seed(RANDOM_SEED)
model = DenseNet121(num_classes=NUM_CLASSES, grayscale=grayscale).to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE)

In [14]:
# Training
def compute_accuracy(model, data_loader, device):
  correct_pred, num_examples = 0., 0
  model.eval()
  for i, (features, targets) in enumerate(data_loader):

    features = features.to(device)
    targets = targets.to(device)

    logits, probas = model(features)
    _, predicted_labels = torch.max(probas, 1)
    num_examples += targets.size(0)
    assert predicted_labels.size() == targets.size()

    correct_pred += (predicted_labels==targets).sum()
  return correct_pred / num_examples * 100

In [15]:
start_time = time.time()

for epoch in range(NUM_EPOCHS):
  model.train()
  
  for batch_idx, (features, targets) in enumerate(train_loader):

    features = features.to(device)
    targets = targets.to(device)

    # FORWARD & BACKPROP
    logits, probas = model(features)
    cost = F.cross_entropy(logits, targets)
    optimizer.zero_grad()

    cost.backward()

    # UPDATE MODEL PARAMETERS
    optimizer.step()

    if batch_idx % 150 == 0:
      print("Batch: %04d/%04d || Epoch: %04d/%04d || Cost: %.3f" % (batch_idx, len(train_loader), epoch+1, NUM_EPOCHS, cost.item()))
    
  model.eval()
  with torch.set_grad_enabled(False):
    train_acc = compute_accuracy(model, train_loader, device)
    valid_acc = compute_accuracy(model, valid_loader, device)

    print("Train Accuracy: %.2f" % (train_acc))
    print("Valid Accuracy: %.2f" % (valid_acc))
  elapsed_time = (time.time() - start_time) / 60
  print("Epoch Elapsed Time: ", elapsed_time, " mins")
elapsed_time = (time.time() - start_time) / 60
print("Total Training Time: ", elapsed_time, " mins")

Batch: 0000/0375 || Epoch: 0001/0020 || Cost: 2.337
Batch: 0150/0375 || Epoch: 0001/0020 || Cost: 1.527
Batch: 0300/0375 || Epoch: 0001/0020 || Cost: 1.262
Train Accuracy: 58.92
Valid Accuracy: 57.15
Epoch Elapsed Time:  0.9925715287526449  mins
Batch: 0000/0375 || Epoch: 0002/0020 || Cost: 1.101
Batch: 0150/0375 || Epoch: 0002/0020 || Cost: 0.997
Batch: 0300/0375 || Epoch: 0002/0020 || Cost: 0.851
Train Accuracy: 66.61
Valid Accuracy: 65.20
Epoch Elapsed Time:  2.0043345212936403  mins
Batch: 0000/0375 || Epoch: 0003/0020 || Cost: 0.844
Batch: 0150/0375 || Epoch: 0003/0020 || Cost: 0.683
Batch: 0300/0375 || Epoch: 0003/0020 || Cost: 0.902
Train Accuracy: 61.20
Valid Accuracy: 59.60
Epoch Elapsed Time:  3.035874553521474  mins
Batch: 0000/0375 || Epoch: 0004/0020 || Cost: 0.587
Batch: 0150/0375 || Epoch: 0004/0020 || Cost: 0.620
Batch: 0300/0375 || Epoch: 0004/0020 || Cost: 0.717
Train Accuracy: 76.72
Valid Accuracy: 70.60
Epoch Elapsed Time:  4.07206107378006  mins
Batch: 0000/0375 ||

In [16]:
# Testing Set
model.eval()
with torch.set_grad_enabled(False):
  test_acc = compute_accuracy(model, test_loader, device)
  print("Test Accuracy: %.2f" % (test_acc.item()))

Test Accuracy: 73.53
